In [ ]:
pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 63.1 MB/s eta 0:00:00


In [ ]:
pip install datasets

In [ ]:
from transformers import Trainer, training_args

In [ ]:
import pdfplumber
def pdf_to_text(pdf_path):
  with pdfplumber.open(pdf_path) as pdf:
    text = "\n".join([page.extract_text() for page in pdf.pages])
  return text

In [ ]:
import re
def clean_text(text):
  text = re.sub(r'Page \d+ of \d+', '', text)
  text = re.sub(r'\s+', ' ', text).strip()
  return text


In [ ]:
from hashlib import sha256
def deduplicate(texts):
  seen = set()
  unique_texts = []
  for text in texts:
    hash_id = sha256(text.encode()).hexdigest()
    if hash_id not in seen:
      seen.add(hash_id)
      unique_texts.append(text)
  return unique_texts

In [ ]:
def chunk_text(text, max_tokens=512):
    sentences = text.split('. ')
    chunks = []
    current_chunk = []
    current_length = 0
    for sent in sentences:
        sent_tokens = len(sent.split())  # Approximate token count
        if current_length + sent_tokens > max_tokens:
            chunks.append(". ".join(current_chunk))
            current_chunk = [sent]
            current_length = sent_tokens
        else:
            current_chunk.append(sent)
            current_length += sent_tokens
    return chunks

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

path_pdf = '/content/drive/My Drive/req_books/Andrew S. Tanenbaum - Computer Networks.pdf'


Mounted at /content/drive


In [ ]:
text_from_pdf = pdf_to_text(path_pdf)


In [ ]:
cleaning_text = clean_text(text_from_pdf)
# print(cleaning_text)

In [ ]:
chunks = chunk_text(cleaning_text)
# print(chunking)

In [ ]:
print(chunks)

['This page intentionally left blank COMPUTER NETWORKS FIFTH EDITION This page intentionally left blank COMPUTER NETWORKS FIFTH EDITION ANDREW S. TANENBAUM Vrije Universiteit Amsterdam, The Netherlands DAVID J. WETHERALL University of Washington Seattle, WA PRENTICE HALL Boston Columbus Indianapolis NewYork SanFrancisco UpperSaddleRiver Amsterdam CapeTown Dubai London Madrid Milan Paris Montreal Toronto Delhi MexicoCity Sao Paulo Sydney HongKong Seoul Singapore Tapei Tokyo Editorial Director: Marcia Horton Art Director: Linda Knowles Editor-in-Chief: Michael Hirsch Cover Designer: Susan Paradise Executive Editor: Tracy Dunkelberger Cover Illustration: Jason Consalvo Assistant Editor: Melinda Haggerty Interior Design: Andrew S. Tanenbaum Editorial Assistant: Allison Michael AV Production Project Manager: Vice President, Marketing: Patrice Jones Gregory L. Dulles Marketing Manager: Yezan Alayan Interior Illustrations: Laserwords, Inc. Marketing Coordinator: Kathryn Ferranti Media Editor:

In [ ]:
with open("corpus.txt", "w") as f:
    f.write("\n".join(chunks))

In [ ]:
from tokenizers import Tokenizer, models, trainers
# Train a BPE tokenizer on your corpus
tokenizer = Tokenizer(models.BPE())
trainer = trainers.BpeTrainer(special_tokens=["[PAD]", "[UNK]"])
tokenizer.train(files=["corpus.txt"], trainer=trainer)
# Save tokenizer
tokenizer.save("custom_tokenizer.json")

In [ ]:
from datasets import load_dataset
from transformers import PreTrainedTokenizerFast

# Load a dataset (e.g., text file or Hugging Face dataset)
dataset = load_dataset("text", data_files={"train": "corpus.txt"})

# Wrap your tokenizer in a Hugging Face-compatible interface
fast_tokenizer = PreTrainedTokenizerFast(tokenizer_file="custom_tokenizer.json")
fast_tokenizer.add_special_tokens({"pad_token": "[PAD]", "unk_token": "[UNK]"})

# Tokenize the dataset
def tokenize_fn(examples):
    inputs = fast_tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )
    inputs["labels"] = inputs["input_ids"].copy()  # Required for training
    return inputs


# Now use it as train_dataset
tokenized_dataset = dataset.map(tokenize_fn, batched=True)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"]
)




AttributeError: module 'transformers.training_args' has no attribute 'batch_eval_metrics'

In [ ]:
trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


Step,Training Loss
500,9.394500


TrainOutput(global_step=528, training_loss=9.384563561641809, metrics={'train_runtime': 4711.9451, 'train_samples_per_second': 0.448, 'train_steps_per_second': 0.112, 'total_flos': 137766223872000.0, 'train_loss': 9.384563561641809, 'epoch': 3.0})

In [ ]:
from transformers import AutoModelForCausalLM, PreTrainedTokenizerFast

from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Fine-tune it with your dataset, then save it properly using `trainer.save_model()`
trainer.save_model("results")  # Saves model, config, tokenizer

# Load the fine-tuned model and tokenizer
model = AutoModelForCausalLM.from_pretrained("/content/results")
tokenizer = PreTrainedTokenizerFast(tokenizer_file="custom_tokenizer.json")
tokenizer.add_special_tokens({"pad_token": "[PAD]", "unk_token": "[UNK]"})

model.eval()  # Set to evaluation mode

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

AttributeError: 'tokenizers.trainers.BpeTrainer' object has no attribute 'save_model'

In [ ]:
# Example prompt from your domain (e.g., computer networks)
prompt = input("Enter text :")

# Tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# Generate text
outputs = model.generate(
    **inputs,
    max_length=100,             # Maximum tokens to generate
    num_return_sequences=1,     # Number of completions
    temperature=0.7,            # Lower = deterministic, higher = creative
    do_sample=True,             # Enable sampling
    top_k=50,                   # Top-k sampling
    pad_token_id=tokenizer.pad_token_id  # Critical for padding
)

# Decode and print
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

Enter text :osi layer


TypeError: 'tokenizers.Tokenizer' object is not callable

In [ ]:
# Example prompt from your domain (e.g., computer networks)
prompt = "The OSI model consists of"

# Tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# Generate text
outputs = model.generate(
    **inputs,
    max_length=100,             # Maximum tokens to generate
    num_return_sequences=1,     # Number of completions
    temperature=0.7,            # Lower = deterministic, higher = creative
    do_sample=True,             # Enable sampling
    top_k=50,                   # Top-k sampling
    pad_token_id=tokenizer.pad_token_id  # Critical for padding
)

# Decode and print
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

NameError: name 'tokenizer' is not defined

In [ ]:
# from transformers import GPT2Config, GPT2LMHeadModel, Trainer, TrainingArguments
# # Initialize model
# config = GPT2Config(vocab_size=tokenizer.get_vocab_size(), n_ctx=512)
# model = GPT2LMHeadModel(config)
# # Define training args
# training_args = TrainingArguments(
#     output_dir="./results",
#     per_device_train_batch_size=4,
#     num_train_epochs=3,
#     logging_dir="./logs",
# )
# # Train
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_dataset,  # Preprocessed dataset
# )
# trainer.train()
from tokenizers import Tokenizer

tokenizer = Tokenizer.from_file("custom_tokenizer.json")
output = tokenizer.encode("Your text to tokenize here")
print("Tokens:", output.tokens)
print("IDs:", output.ids)



Tokens: ['Y', 'our ', 'text ', 'to ', 'token', 'ize ', 'here']
IDs: [57, 885, 1233, 175, 5580, 3072, 7243]
